In [1]:
import cv2
import torch
from ultralytics import YOLO
import numpy as np
import math
from numpy import random
from IPython.display import HTML
import torchvision.models as torch_models
from base64 import b64encode
import os
from IPython.display import Video
from utils import *
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from cls_detection import *
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else device
print(device)

preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                        std=[0.229, 0.224, 0.225]),
])



cuda


In [3]:
modelPath = "cls_chkpoint_resnet18/checkpoint_2024-01-05-14-22_lr_0.0005614992795738434_batch_64/checkpoint_2024-01-05-14-22_lr_0.0005614992795738434_batch_64/checkpoint_epoch_65.pth"
batch_size = 128 
conf = 0.3

cls_model = load_resnet18(modelPath)
#resnet50 = load_resnet50(resnet50_path)

In [ ]:
timestamps_all_made = inference_video(model = model,
                                      cls_model = cls_model,
                                      video_path = "video_test_dataset/all_made.mp4",
                                      save_result_vid = True,
                                      display_result = True,
                                      show_progress=True,
                                      cls_conf_threshold=0.3,
                                      device="cuda",
                                      show_score_prob=True,
                                      detect_conf_threshold=0.7,
                                      batch_mode=True,
                                      batch_size=128,
                                      ffmpeg_path="ffmpeg",
                                      num_buffer_frames=15
                                      )
print(len(timestamps_all_made))

In [5]:
yolo_model_path = "weights/detect_large_v10.pt"
model = YOLO(yolo_model_path)
classNames = ['basketball', 'hoop', 'made', 'person']

inference_video(model = model,
                cls_model = cls_model,
                video_path = "video_test_dataset/1/made_14.mp4",
                save_result_vid = True,
                display_result = True,
                show_progress=True,
                cls_conf_threshold=0.3,
                device="cuda",
                show_score_prob=True,
                detect_conf_threshold=0.7,
                batch_mode=False,
                batch_size=128,
                ffmpeg_path="ffmpeg"
                )

inference by frame


 34%|███▍      | 63/183 [00:03<00:05, 20.19it/s]


[(2.29, array([     0.3751], dtype=float32))]